# Mistral 7B Try

This notebook is used to try the pipeline without the integration of MongoDB to store the versioning of the dataset

### Dataset and preprocessing

In [10]:
#Load the SQuAD v2 dataset using the Hugging Face datasets library
from datasets import load_dataset

dataset = load_dataset("squad_v2")
print("Number of examples in the dataset:", len(dataset["train"]))
print("First example in the dataset:", dataset["train"][0])

Number of examples in the dataset: 130319
First example in the dataset: {'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}


In [16]:
#Define a function to format the dataset examples into a prompt
#The prompt will include the context, question, and answer
def make_prompt(example):
    context = example["context"]
    question = example["question"]
    answer = example["answers"]["text"][0] if example["answers"]["text"] else "No answer"

    prompt = f"[INST] Given the context, answer the question.\n\nContext: {context}\n\nQuestion: {question} [/INST] {answer}"
    return {"prompt": prompt, "reference": answer}

formatted_dataset = {
    split: dataset[split].map(make_prompt)
    for split in dataset.keys()
}

Map: 100%|██████████| 130319/130319 [00:06<00:00, 19179.64 examples/s]


In [19]:
# Print the first formatted prompt and its reference answer
print("Prompt:\n", formatted_dataset["train"][0]["prompt"])
print("\nReference Answer:\n", formatted_dataset["train"][0]["reference"])

Prompt:
 [INST] Given the context, answer the question.

Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

Question: When did Beyonce start becoming popular? [/INST] in the late 1990s

Reference Answer:
 in the late 1990s


### Tokenizer

In [38]:
from dotenv import load_dotenv
import os

load_dotenv("key.env")
token = os.getenv("HUGGINGFACE_TOKEN")

from huggingface_hub import login
login(token=token)

In [57]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=512)

tokenized = {split: formatted_dataset[split].map(tokenize, batched=True) for split in formatted_dataset.keys()}

Map: 100%|██████████| 11873/11873 [00:01<00:00, 8998.20 examples/s]


### Load Mistral model

In [58]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "mistralai/Mistral-Small-3.1-24B-Base-2503"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False,
    trust_remote_code=True  # <- Add this line to load custom model code
)

OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

### Training

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./mistral-squad2-fullfinetune",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,  # Use fp16 if supported by your GPU
    evaluation_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    tokenizer=tokenizer
)

trainer.train()

### Generate Predictions on Validation